In [1]:
from decouple import config


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')


In [ ]:
# https://qdrant.tech/documentation/quickstart/

In [2]:
from qdrant_client import QdrantClient


qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[]


In [4]:
from fastembed import TextEmbedding


documents = [
    'FastEmbed is lighter than Transformers & Sentence-Transformers.',
    'FastEmbed is supported by and maintained by Qdrant.',
]

In [5]:
embedding_model = TextEmbedding(model_name='BAAI/bge-small-en-v1.5')

Fetching 5 files: 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


In [ ]:
# https://qdrant.tech/documentation/fastembed/fastembed-quickstart/

embeddings = list(embedding_model.embed(documents))
len(embeddings[0])  

384

In [ ]:
from qdrant_client.models import Distance, VectorParams


COLLECTION_NAME = ...

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

In [ ]:
from qdrant_client.models import PointStruct
from uuid import uuid4

operation_info = qdrant_client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=[
        PointStruct(
            id=uuid4(), 
            vector=embedding.tolist(), 
            payload={'text': document}
        )
        for document, embedding in zip(documents, embeddings)
    ],
)

print(operation_info)

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue


search_result = qdrant_client.query_points(
    collection_name=COLLECTION_NAME,
    query=...,
    #query_filter=Filter(
    #    must=[FieldCondition(key='text', match=MatchValue(value=...))]  # metadata
    #),
    with_payload=False,
    limit=3
).points

print(search_result)

In [10]:
from fastembed import SparseTextEmbedding, SparseEmbedding

# https://qdrant.tech/documentation/fastembed/fastembed-splade/

In [ ]:
SparseTextEmbedding.list_supported_models()

In [ ]:
model = SparseTextEmbedding(model_name='prithvida/Splade_PP_en_v1')

documents = [
    'Chandrayaan-3 is India\'s third lunar mission',
    'It aimed to land a rover on the Moon\'s surface - joining the US, China and Russia',
    'The mission is a follow-up to Chandrayaan-2, which had partial success',
    'Chandrayaan-3 will be launched by the Indian Space Research Organisation (ISRO)',
    'The estimated cost of the mission is around $35 million',
    'It will carry instruments to study the lunar surface and atmosphere',
    'Chandrayaan-3 landed on the Moon\'s surface on 23rd August 2023',
    'It consists of a lander named Vikram and a rover named Pragyan similar to Chandrayaan-2. Its propulsion module would act like an orbiter.',
    'The propulsion module carries the lander and rover configuration until the spacecraft is in a 100-kilometre (62 mi) lunar orbit',
    'The mission used GSLV Mk III rocket for its launch',
    'Chandrayaan-3 was launched from the Satish Dhawan Space Centre in Sriharikota',
    'Chandrayaan-3 was launched earlier in the year 2023',
]

In [ ]:
sparse_embeddings = list(model.embed(documents, batch_size=5)) 

In [ ]:
import json
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained('Qdrant/SPLADE_PP_en_v1')

In [ ]:
def get_tokens_and_weights(sparse_embedding: SparseEmbedding, tokenizer: Tokenizer) -> dict:
    token_weight_dict = {}
    
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # Sort the dictionary by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    
    return token_weight_dict

In [ ]:
index = 0
print(json.dumps(get_tokens_and_weights(sparse_embeddings[index], tokenizer), indent=4))